In [ ]:
# evaluate_goodness.py
import json
from typing import Tuple

def evaluate_goodness(json_path: str) -> Tuple[int, dict]:
    """
    Evaluate a .json file containing a recipe JSON according to the rubric:
      a) Correct JSON format: 50 points (0 if invalid JSON)
      b) Presence of metadata keys: 10 points each (max 50)
         keys: 'recipe_name', 'data_provenance', 'macronutrients', 'ingredients', 'instructions'
    Returns (score, breakdown_dict)
    breakdown_dict contains boolean flags and intermediate scores.
    """
    breakdown = {
        "valid_json": False,
        "format_score": 0,
        "keys_present": {
            "recipe_name": False,
            "data_provenance": False,
            "macronutrients": False,
            "ingredients": False,
            "instructions": False
        },
        "keys_score": 0,
        "total_score": 0,
        "error": ""
    }

    # Try to load JSON
    try:
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)
    except Exception as e:
        breakdown["error"] = f"JSON load error: {e}"
        # invalid JSON -> score 0
        breakdown["total_score"] = 0
        return 0, breakdown

    # If we reached here, JSON parsed
    breakdown["valid_json"] = True
    breakdown["format_score"] = 50

    # Check presence of required top-level keys
    keys = ["recipe_name", "data_provenance", "macronutrients", "ingredients", "instructions"]
    key_points = 10
    keys_found = 0
    for k in keys:
        if k in data:
            # additional sanity check: ensure the value is not None (but presence is main requirement)
            breakdown["keys_present"][k] = True
            keys_found += 1
        else:
            breakdown["keys_present"][k] = False

    breakdown["keys_score"] = keys_found * key_points
    breakdown["total_score"] = breakdown["format_score"] + breakdown["keys_score"]
    return breakdown["total_score"], breakdown

if __name__ == "__main__":
    # Example usage: adjust path to your JSON file
    sample_path = "recipe_output.json"
    score, details = evaluate_goodness(sample_path)
    print(f"Goodness score: {score}/100")
    print("Details:")
    print(json.dumps(details, indent=2))
